In [1]:
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel("TESTE.xlsx")

### Agências em Mínimo

In [3]:
def minimo(a):
    if (a['A'] > 0 and a['B'] == 0) or (a['A'] == 0 and a['B'] > 0):
        if a['A'] + a['B'] < 4:
            return 'Minimo'
        else:
            return "Espaco"
    else:
        if a['A'] + a['B'] < 5:
            return "Minimo"
        else:
            return "Espaco"

In [4]:
df['minimo'] = df.apply(minimo, axis=1)

In [5]:
df.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo
0,1,3,0,0.46,0.00,4,0,5,0,Minimo
1,2,4,0,0.30,0.00,4,0,3,0,Espaco
2,3,0,3,0.00,0.47,0,2,0,2,Minimo
3,4,2,0,0.30,0.00,2,0,1,0,Minimo
4,5,3,2,0.40,0.20,3,1,3,1,Espaco


## Separando entre Hall Unico e Misto

In [6]:
def ambiente(a):
    if (a['A'] > 0 and a['B'] == 0) or (a['B'] > 0 and a['A'] == 0):
        return 'Unico'
    else:
        return 'Misto'

In [7]:
df['ambiente'] = df.apply(ambiente, axis=1)

In [8]:
df.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo,ambiente
0,1,3,0,0.46,0.00,4,0,5,0,Minimo,Unico
1,2,4,0,0.30,0.00,4,0,3,0,Espaco,Unico
2,3,0,3,0.00,0.47,0,2,0,2,Minimo,Unico
3,4,2,0,0.30,0.00,2,0,1,0,Minimo,Unico
4,5,3,2,0.40,0.20,3,1,3,1,Espaco,Misto


In [9]:
df_un = df[df['ambiente']=="Unico"]
df_mi = df[df['ambiente']=="Misto"]

# Hall Único

In [10]:
df_un.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo,ambiente
0,1,3,0,0.46,0.00,4,0,5,0,Minimo,Unico
1,2,4,0,0.30,0.00,4,0,3,0,Espaco,Unico
2,3,0,3,0.00,0.47,0,2,0,2,Minimo,Unico
3,4,2,0,0.30,0.00,2,0,1,0,Minimo,Unico
5,6,4,0,0.40,0.20,3,0,2,0,Espaco,Unico


### Status DIM

In [11]:
def status(a):
    if (a['A']+a['B'] < a['A1']+a['B1']):
        return 'Acrescimo'
    elif (a['A']+a['B'] > a['A1']+a['B1']):
        return 'Reducao'
    else:
        return 'Ideal'

In [12]:
df_un['status'] = df_un.apply(status, axis=1)

In [13]:
df_un.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo,ambiente,status
0,1,3,0,0.46,0.00,4,0,5,0,Minimo,Unico,Acrescimo
1,2,4,0,0.30,0.00,4,0,3,0,Espaco,Unico,Ideal
2,3,0,3,0.00,0.47,0,2,0,2,Minimo,Unico,Reducao
3,4,2,0,0.30,0.00,2,0,1,0,Minimo,Unico,Ideal
5,6,4,0,0.40,0.20,3,0,2,0,Espaco,Unico,Reducao


### Regras Minimos

In [14]:
df_un['percent_mudanca_1'] = abs(((df_un['A1'] + df_un['B1']) / (df_un['A'] + df_un['B']))-1)
df_un['percent_mudanca_2'] = abs(((df_un['A2'] + df_un['B2']) / (df_un['A1'] + df_un['B1']))-1)

In [15]:
df_un.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo,ambiente,status,percent_mudanca_1,percent_mudanca_2
0,1,3,0,0.46,0.00,4,0,5,0,Minimo,Unico,Acrescimo,0.333333,0.250000
1,2,4,0,0.30,0.00,4,0,3,0,Espaco,Unico,Ideal,0.000000,0.250000
2,3,0,3,0.00,0.47,0,2,0,2,Minimo,Unico,Reducao,0.333333,0.000000
3,4,2,0,0.30,0.00,2,0,1,0,Minimo,Unico,Ideal,0.000000,0.500000
5,6,4,0,0.40,0.20,3,0,2,0,Espaco,Unico,Reducao,0.250000,0.333333


In [16]:
def ajuste_minimo_1(a):
    if a['percent_mudanca_1'] > 0.4:
        if a['status'] == 'Acrescimo':
            return int((a['A']+a['B'])*0.4)
        elif a['status'] == 'Reducao':
            return -(int((a['A']+a['B'])*0.4))
        else:
            return 0
    else:
        return ((a['A1']+a['B1']) - (a['A']+a['B']))

In [17]:
df_un['delta'] = df_un.apply(ajuste_minimo_1, axis=1)

In [18]:
def ajuste_minimo_2(a):
    if a['percent_mudanca_2'] > 0.4:
        if a['status'] == 'Acrescimo':
            return int((a['A1']+a['B1'])*0.4)
        elif a['status'] == 'Reducao':
            return -(int((a['A1']+a['B1'])*0.4))
        else:
            return 0
    else:
        return ((a['A2']+a['B2']) - (a['A1']+a['B1']))

In [19]:
df_un['delta_1'] = df_un.apply(ajuste_minimo_2, axis=1)

In [20]:
df_un.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo,ambiente,status,percent_mudanca_1,percent_mudanca_2,delta,delta_1
0,1,3,0,0.46,0.00,4,0,5,0,Minimo,Unico,Acrescimo,0.333333,0.250000,1,1
1,2,4,0,0.30,0.00,4,0,3,0,Espaco,Unico,Ideal,0.000000,0.250000,0,-1
2,3,0,3,0.00,0.47,0,2,0,2,Minimo,Unico,Reducao,0.333333,0.000000,-1,0
3,4,2,0,0.30,0.00,2,0,1,0,Minimo,Unico,Ideal,0.000000,0.500000,0,0
5,6,4,0,0.40,0.20,3,0,2,0,Espaco,Unico,Reducao,0.250000,0.333333,-1,-1


In [21]:
def ajustado_1(a):
    if a['A']> 0 and a['B'] == 0:
        return a['A'] + a['delta'],a['B']
    elif a['A'] == 0 and a['B'] > 0:
        return a['A'],a['B'] + a['delta']

In [22]:
df_un['A1_F'],df_un['B1_F'] = zip(*df_un.apply(ajustado_1, axis=1))

In [23]:
df_un.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo,ambiente,status,percent_mudanca_1,percent_mudanca_2,delta,delta_1,A1_F,B1_F
0,1,3,0,0.46,0.00,4,0,5,0,Minimo,Unico,Acrescimo,0.333333,0.250000,1,1,4,0
1,2,4,0,0.30,0.00,4,0,3,0,Espaco,Unico,Ideal,0.000000,0.250000,0,-1,4,0
2,3,0,3,0.00,0.47,0,2,0,2,Minimo,Unico,Reducao,0.333333,0.000000,-1,0,0,2
3,4,2,0,0.30,0.00,2,0,1,0,Minimo,Unico,Ideal,0.000000,0.500000,0,0,2,0
5,6,4,0,0.40,0.20,3,0,2,0,Espaco,Unico,Reducao,0.250000,0.333333,-1,-1,3,0


In [24]:
def ajustado_2(a):
    if a['A1']> 0 and a['B1'] == 0:
        return a['A1'] + a['delta_1'],a['B1']
    elif a['A1'] == 0 and a['B1'] > 0:
        return a['A1'],a['B1'] + a['delta_1']

In [25]:
df_un['A2_F'],df_un['B2_F'] = zip(*df_un.apply(ajustado_2, axis=1))

In [26]:
df_un.head()

,X,A,B,F1,F2,A1,B1,A2,B2,minimo,ambiente,status,percent_mudanca_1,percent_mudanca_2,delta,delta_1,A1_F,B1_F,A2_F,B2_F
0,1,3,0,0.46,0.00,4,0,5,0,Minimo,Unico,Acrescimo,0.333333,0.250000,1,1,4,0,5,0
1,2,4,0,0.30,0.00,4,0,3,0,Espaco,Unico,Ideal,0.000000,0.250000,0,-1,4,0,3,0
2,3,0,3,0.00,0.47,0,2,0,2,Minimo,Unico,Reducao,0.333333,0.000000,-1,0,0,2,0,2
3,4,2,0,0.30,0.00,2,0,1,0,Minimo,Unico,Ideal,0.000000,0.500000,0,0,2,0,2,0
5,6,4,0,0.40,0.20,3,0,2,0,Espaco,Unico,Reducao,0.250000,0.333333,-1,-1,3,0,2,0


In [27]:
df_un = df_un[['X','A','B','F1','F2','A1_F','B1_F','A2_F','B2_F','status','minimo']]

In [28]:
df_un.head()

,X,A,B,F1,F2,A1_F,B1_F,A2_F,B2_F,status,minimo
0,1,3,0,0.46,0.00,4,0,5,0,Acrescimo,Minimo
1,2,4,0,0.30,0.00,4,0,3,0,Ideal,Espaco
2,3,0,3,0.00,0.47,0,2,0,2,Reducao,Minimo
3,4,2,0,0.30,0.00,2,0,2,0,Ideal,Minimo
5,6,4,0,0.40,0.20,3,0,2,0,Reducao,Espaco


### Acrescimo em Hall Unico

In [29]:
df_un_A = df_un[df_un['status']=="Acrescimo"]

In [30]:
#pegar fila do hall
df_un_A['F'] = df_un_A[["F1", "F2"]].max(axis=1)

In [31]:
df_un_A.head()

,X,A,B,F1,F2,A1_F,B1_F,A2_F,B2_F,status,minimo,F
0,1,3,0,0.46,0.0,4,0,5,0,Acrescimo,Minimo,0.46


In [32]:
def comp_fila(a):
    if a['F'] > 0.45:
        return 'Acrescimo_validado_fila'
    else:
        return 'Acrescimo_N_validado_fila'

In [33]:
df_un_A['status_dim_fila'] = df_un_A.apply(comp_fila, axis=1)

In [34]:
df_un_A.head()

,X,A,B,F1,F2,A1_F,B1_F,A2_F,B2_F,status,minimo,F,status_dim_fila
0,1,3,0,0.46,0.0,4,0,5,0,Acrescimo,Minimo,0.46,Acrescimo_validado_fila


In [35]:
def dim_final(a):
    if a['status_dim_fila'] == 'Acrescimo_validado_fila':
        return min(a['A1_F'],a['A2_F']),min(a['B1_F'],a['B2_F'])
    else:
        return a['A'], a['B']

In [36]:
df_un_A['A_F'],df_un_A['B_F'] = zip(*df_un_A.apply(dim_final, axis=1))

In [37]:
df_un_A.head()

,X,A,B,F1,F2,A1_F,B1_F,A2_F,B2_F,status,minimo,F,status_dim_fila,A_F,B_F
0,1,3,0,0.46,0.0,4,0,5,0,Acrescimo,Minimo,0.46,Acrescimo_validado_fila,4,0


### Ideal em Hall Unico

In [38]:
df_un_I = df_un[df_un['status']=="Ideal"]

In [39]:
df_un_I.head()

,X,A,B,F1,F2,A1_F,B1_F,A2_F,B2_F,status,minimo
1,2,4,0,0.30,0.0,4,0,3,0,Ideal,Espaco
3,4,2,0,0.30,0.0,2,0,2,0,Ideal,Minimo
6,7,5,0,0.55,0.3,5,0,6,0,Ideal,Espaco


In [40]:
df_un_I['A_F'] = df_un_I['A']
df_un_I['B_F'] = df_un_I['B']

### Reducao em Hall Unico

In [41]:
df_un_R = df_un[df_un['status']=="Reducao"]

In [42]:
#pegar fila do hall
df_un_R['F'] = df_un_R[["F1", "F2"]].max(axis=1)

In [43]:
df_un_R

,X,A,B,F1,F2,A1_F,B1_F,A2_F,B2_F,status,minimo,F
2,3,0,3,0.0,0.47,0,2,0,2,Reducao,Minimo,0.47
5,6,4,0,0.4,0.20,3,0,2,0,Reducao,Espaco,0.40


In [44]:
def reduz(a):
    if a['minimo'] == "Minimo":
        return a['A'],a['B']
    elif a['F'] > 0.4:
        return a['A'],a['B']
    else:
        return a['A1_F'],a['B1_F']

In [45]:
df_un_R['A_F'],df_un_R['B_F'] = zip(*df_un_R.apply(reduz, axis=1))

In [46]:
df_un_R.head()

,X,A,B,F1,F2,A1_F,B1_F,A2_F,B2_F,status,minimo,F,A_F,B_F
2,3,0,3,0.0,0.47,0,2,0,2,Reducao,Minimo,0.47,0,3
5,6,4,0,0.4,0.20,3,0,2,0,Reducao,Espaco,0.40,3,0
